# 1.Importing required modules

In [ ]:
import os
import pandas as pd
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from pymongo import MongoClient
os.chdir("E:\\sample proj")
import glob
import shutil
from datetime import datetime
from time import sleep
import sys
import logging
logging.basicConfig(filename="logfile.log", level=logging.INFO)

# 2.Searching for files

In [2]:
while True:
    path,dirs,files=next(os.walk("E:\\sample proj"))
    file_count=len(files)
    logging.info('searching for files')
    if file_count > 6:
        files_cy  = glob.glob('*cy*.csv')
        files_ytd = glob.glob('*ytd*.csv')
        logging.info('files found moving to next process')
        logging.info('spliting files')
        break
        
    else:
        logging.info('waiting for files')
        sleep(20)

KeyboardInterrupt: 

# 3.Connect to server & database

In [ ]:
try: 
    conn = MongoClient() 
    print("Connected successfully!!!") 
    logging.info('connecting to database')
except:   
    print("Could not connect to MongoDB")
    logging.info('error while coonecting to database')
    

In [5]:
db = conn.empdb

# 4.Separating the files and insert into mongo colletions

In [ ]:
for csv in files_cy:
    
    if 'cy_2017' in csv:
        # reading the csv file
        
        df = pd.read_csv(csv)
        
        #connecting to table
        collection = db.emp_cy_2017
        
        #removing previous data
        collection.remove()
        
        #updating recent data
        collection.insert(df.to_dict('records'))
        
    elif 'cy_2018' in csv:
        df = pd.read_csv(csv)
        collection = db.emp_cy_2018
        collection.remove()
        collection.insert(df.to_dict('records'))
        
    elif 'cy_2019' in csv:
        df = pd.read_csv(csv)
        collection = db.emp_cy_2019
        collection.remove()
        collection.insert(df.to_dict('records'))
        
logging.info('inserted CY data into database')        

In [7]:
for csv in files_ytd:
    
    if 'ytd_2017' in csv:
        df = pd.read_csv(csv)
        collection = db.emp_ytd_2017
        collection.remove()
        collection.insert(df.to_dict('records'))
        
    elif 'ytd_2018' in csv:
        df = pd.read_csv(csv)
        collection = db.emp_ytd_2018
        collection.remove()
        collection.insert(df.to_dict('records'))
        
    elif 'ytd_2019' in csv:
        df = pd.read_csv(csv)
        collection = db.emp_ytd_2019
        collection.remove()
        collection.insert(df.to_dict('records'))

logging.info('inserted YTD data into database') 

C:\Users\SAIKRISHNA\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  
C:\Users\SAIKRISHNA\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  import sys
C:\Users\SAIKRISHNA\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  if sys.path[0] == '':
C:\Users\SAIKRISHNA\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  del sys.path[0]
C:\Users\SAIKRISHNA\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
C:\Users\SAIKRISHNA\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


# 5.backup the files

In [ ]:
today = datetime.now()
name = today.strftime('%Y%m%d')
path = os.getcwd()
shutil.make_archive(name,'zip',path)
logging.info('creating backup for files')

# 6. mail

In [9]:
li = ["navyakrishnatest@gmail.com","Srikarreddy0007@gmail.com","mschary07@gmail.com"]
msg=MIMEMultipart()
message="Six files updated, inserted into six collections"
msg.attach(MIMEText(message,'plain'))
msg.attach(MIMEText(open("E:\\sample proj\\logfile.log").read()))


In [ ]:
for i in range(len(li)):
    s = smtplib.SMTP('smtp.gmail.com',587)
    s.starttls()
    s.login("navyakrishnatest@gmail.com", "xxxxxxx")
    msg['Subject']= "Files updated successfully"
    s.sendmail("navyakrishnatest@gmail.com", li[i], msg.as_string())
    s.quit()
